# Pointcloud usage tutorial

### This tutorials show how to plot pointcloud in the world coordinate and ego coordinate.
#### 1.Import required python modules and load sequence data.

In [1]:
import pandaset
import os

# load dataset
dataset = pandaset.DataSet('../../test/data')
seq002 = dataset["002"]
seq002.load_lidar().load_semseg()

#### 2.Plot LIDAR points for Pandar64 and PandarGT in the world coordinate.
-  Plot Pandar64 pointcloud by points' labels ```d=0``` colorized as blue
-  Plot PandarGT pointcloud by points' labels ```d=1``` colorized as red

In [4]:
import open3d as o3d
from open3d import JVisualizer


seq_idx = 40
print(seq002.lidar[seq_idx].columns)
points = seq002.lidar[seq_idx].to_numpy()
# get Pandar64 points

pandar64_points = points[(points[:, -1] == 0)]
print("Pandar64 has points: ", pandar64_points.shape)

pandarGT_points = points[(points[:, -1] == 1)]
print("PandarGT has points: ", pandarGT_points.shape)

axis_pcd = o3d.create_mesh_coordinate_frame(size=2.0, origin=[0, 0, 0])

p64_pc = o3d.geometry.PointCloud()
p64_pc.points = o3d.utility.Vector3dVector(pandar64_points[:, :3])
p64_pc.colors = o3d.Vector3dVector([[0, 0, 1] for _ in range(pandar64_points.shape[0])])

gt_pc = o3d.geometry.PointCloud()
gt_pc.points = o3d.utility.Vector3dVector(pandarGT_points[:, :3])
gt_pc.colors = o3d.Vector3dVector([[1, 0, 0] for _ in range(pandarGT_points.shape[0])])

o3d.visualization.draw_geometries([axis_pcd, p64_pc, gt_pc], window_name="world frame")


Index(['x', 'y', 'z', 'i', 't', 'd'], dtype='object')
Pandar64 has points:  (106207, 6)
PandarGT has points:  (62089, 6)


#### 3.Plot LIDAR points for Pandar64 and PandarGT in the ego coordinate.
- Use geometry.lidar_points_to_ego to transform points in the world coordinate to the ego coordinate.
- ***geometry.lidar_points_to_ego***
    - input
        - ***lidar_points***(np.array(\[N, 3\])): lidar points in the world coordinates.
        - ***lidar_pose***: pose in the world coordinates for one camera in one frame.
    - output
        - ***lidar_points_in_ego***(np.array(\[N, 2\])): lidar points in the ego coordinates.

In [5]:
import open3d as o3d
from open3d import JVisualizer

from pandaset import geometry

ego_pandar64_points = geometry.lidar_points_to_ego(pandar64_points[:, :3], seq002.lidar.poses[seq_idx])
p64_pc = o3d.geometry.PointCloud()
p64_pc.points = o3d.utility.Vector3dVector(ego_pandar64_points)
p64_pc.colors = o3d.Vector3dVector([[0, 0, 1] for _ in range(pandar64_points.shape[0])])

ego_pandarGT_points = geometry.lidar_points_to_ego(pandarGT_points[:, :3], seq002.lidar.poses[seq_idx])
gt_pc = o3d.geometry.PointCloud()
gt_pc.points = o3d.utility.Vector3dVector(ego_pandarGT_points)
gt_pc.colors = o3d.Vector3dVector([[1, 0, 0] for _ in range(pandarGT_points.shape[0])])

o3d.visualization.draw_geometries([axis_pcd, p64_pc, gt_pc], window_name="ego frame")